In [3]:
import pandas as pd
import geopandas as gpd
import matplotlib.pyplot as plt
import osmnx as ox

# handle filepaths
import os
import configparser

In [4]:
#set filepaths
project_root = os.path.abspath(os.getcwd())
root = os.path.abspath(os.path.join(os.getcwd(), '..', '..'))
config_file =  os.path.join(root, 'config.cfg')

# geodata for shapefiles
config = configparser.ConfigParser()
config.read(config_file)
gdata_root = config['geodata']['path']

## Load data

### London shapefiles

In [5]:
# get London borough shapefile
london = gpd.GeoDataFrame.from_file(os.path.join(gdata_root, 'london-shapefiles\\London_Borough_Excluding_MHW.shp'))
london = london.to_crs('EPSG:4326')

# convert to correct CRS to have units in meters in the UK
_crs = "EPSG:32630"
london_reprojected = london.to_crs(_crs)

### Funding bids

In [106]:
funding = pd.read_csv("transport_bids_by_borough.csv", skiprows=2, usecols=[0,1,2,3], header=None, names=["GSS_CODE","borough", "cycling_bid", "cycling_funding"])

### TfL cycle routes

In [56]:
# load TFL cycle routes
tfl_cycle_routes = gpd.GeoDataFrame.from_file('TfL_CycleRoutes.json')

### OSM data

Gets data from OSM using `osmnx` and save in csv files to load in for future runs

#### OSM road stats

In [ ]:
road_stats = {"NAME": [],
              "OSM_street_length_total": []
              }

for i in range(len(london)):
    roads = ox.graph_from_polygon(london["geometry"][i],network_type='drive')
    basic_stats = ox.stats.basic_stats(roads, area=None)
    road_stats["NAME"].append(london["NAME"][i])
    road_stats["OSM_street_length_total"].append(basic_stats["street_length_total"])
    
road_stats_df = pd.DataFrame.from_dict(road_stats)

In [6]:
# save output
road_stats_df.to_csv("OSM_road_stats_by_borough.csv", index=False)

In [72]:
# load preprocessed csv file
road_stats_df = pd.read_csv("OSM_road_stats_by_borough.csv")

#### OSM cycle routes

Useful code in [this tutorial](https://williamthyer.github.io/posts/2021/4/bike_networks/) and 
the [original Github Thread](https://github.com/gboeing/osmnx/issues/151)

Example notebooks in docs: https://github.com/gboeing/osmnx-examples/blob/main/notebooks/08-custom-filters-infrastructure.ipynb

In [6]:
# Configuring osmnx
ox.settings.useful_tags_way += ["cycleway"]

def get_cycleways_polygon(polygon):    
    # Querying for roads and bike trails
    cycleways = ox.graph_from_polygon(polygon, network_type='bike', simplify=True)
    
    # Finding all non-cycleways in the network
    non_cycleways = [(u, v, k) for  u, v, k, d  in  cycleways.edges(keys=True, data=True) if  not ('cycleway'  in  d  or  d['highway']=='cycleway')]
   
    # Remove non-cycleways and isolated nodes
    cycleways.remove_edges_from(non_cycleways)
    cycleways = ox.utils_graph.remove_isolated_nodes(cycleways)
    
    return cycleways

In [13]:
cycleways_df = []
cycle_stats = {"NAME": [],
              "OSM_cycleways_length_total": [],
              }

for i in range(len(london)):
    # get cycleways for each borough
    polygon = london["geometry"][i]
    cycleways = get_cycleways_polygon(polygon)

    # get basic stats
    basic_stats = ox.stats.basic_stats(cycleways, area=None)
    cycle_stats["NAME"].append(london["NAME"][i])
    cycle_stats["OSM_cycleways_length_total"].append(basic_stats["street_length_total"])

    # turn cycleways into gdf
    cycleways_df_single = ox.utils_graph.graph_to_gdfs(cycleways, nodes=False, edges=True)
    cols = ['highway',  'cycleway', 'length', 'geometry']
    cycleways_df_single = cycleways_df_single[cols].reset_index(drop=True)
    cycleways_df_single["NAME"] = london["NAME"][i]
    cycleways_df.append(cycleways_df_single)

cycleways_df = pd.concat(cycleways_df, ignore_index=True)
cycleways_stats_df = pd.DataFrame.from_dict(cycle_stats)

In [11]:
# save outputs
cycleways_df = cycleways_df.rename_geometry('WKT')
cycleways_df.to_csv("OSM_cycleways_by_borough.csv", index=False)
cycleways_stats_df.to_csv("OSM_cycleways_stats_by_borough.csv", index=False)

In [71]:
# load preprocessed csv file
cycleways_stats_df = pd.read_csv("OSM_cycleways_stats_by_borough.csv")
cycleways_gdf = gpd.read_file("OSM_cycleways_by_borough.csv")
cycleways_gdf = cycleways_gdf.set_crs('EPSG:4326')

## Process data

In [108]:
gdf = london[['NAME', 'GSS_CODE', 'HECTARES', 'NONLD_AREA', 'ONS_INNER', 'geometry']]

### Funding

In [109]:
gdf = pd.merge(gdf, funding[["GSS_CODE","cycling_bid", "cycling_funding"]], left_on="GSS_CODE", right_on="GSS_CODE", how="left")

### OSM stats

In [110]:
gdf = pd.merge(gdf, road_stats_df, left_on="NAME", right_on="NAME", how="left")
gdf = pd.merge(gdf, cycleways_stats_df, left_on="NAME", right_on="NAME", how="left")

### TfL routes length

In [111]:
tfl_cycle_routes_total_length = []
tfl_cycle_routes_open_length = []
tfl_cycle_routes_planned_length = []

for i in range(len(london)):
    routes_clipped = gpd.clip(tfl_cycle_routes, london["geometry"][i], keep_geom_type=True)
    tfl_cycle_routes_total_length.append(routes_clipped["Shape_Leng"].sum())

    open_routes_clipped = gpd.clip(tfl_cycle_routes[tfl_cycle_routes["Status"]=="Open"], london["geometry"][i], keep_geom_type=True)
    tfl_cycle_routes_open_length.append(open_routes_clipped["Shape_Leng"].sum())

    planned_routes_clipped = gpd.clip(tfl_cycle_routes[tfl_cycle_routes["Status"]!="Open"], london["geometry"][i], keep_geom_type=True)
    tfl_cycle_routes_planned_length.append(planned_routes_clipped["Shape_Leng"].sum())

# add to gdf
gdf.loc[:,"tfl_cycle_routes_total_length"] = tfl_cycle_routes_total_length
gdf.loc[:,"tfl_cycle_routes_open_length"] = tfl_cycle_routes_open_length
gdf.loc[:,"tfl_cycle_routes_planned_length"] = tfl_cycle_routes_planned_length

### Calculate ratios

In [112]:
gdf

,NAME,GSS_CODE,HECTARES,NONLD_AREA,ONS_INNER,geometry,cycling_bid,cycling_funding,OSM_street_length_total,OSM_intersection_count,OSM_cycleways_length_total,tfl_cycle_routes_total_length,tfl_cycle_routes_open_length,tfl_cycle_routes_planned_length
0,Kingston upon Thames,E09000021,3726.117,0.000,F,"POLYGON ((-0.33066 51.32901, -0.33057 51.32909...",0,0,348442.337,2086,36092.105,17985.913505,16381.641919,1604.271586
1,Croydon,E09000008,8649.441,0.000,F,"POLYGON ((-0.06399 51.31864, -0.06405 51.31861...",0,80,794682.782,4630,38650.448,0.000000,0.000000,0.000000
2,Bromley,E09000006,15013.487,0.000,F,"POLYGON ((0.01216 51.29960, 0.01199 51.29979, ...",21,5,911900.591,4699,38665.652,4876.804524,0.000000,4876.804524
3,Hounslow,E09000018,5658.541,60.755,F,"POLYGON ((-0.24454 51.48870, -0.24466 51.48868...",804,225,528340.936,3204,68563.904,19526.447194,14234.219367,5292.227827
4,Ealing,E09000009,5554.428,0.000,F,"POLYGON ((-0.41181 51.53408, -0.41186 51.53413...",540,115,593591.492,3535,75333.218,44393.719253,30877.084811,13516.634442
5,Havering,E09000016,11445.735,210.763,F,"POLYGON ((0.15872 51.51219, 0.15873 51.51224, ...",200,50,700001.907,3676,51563.295,0.000000,0.000000,0.000000
6,Hillingdon,E09000017,11570.063,0.000,F,"POLYGON ((-0.40405 51.61318, -0.40386 51.61230...",165,165,821816.786,4620,67717.875,23259.475437,23259.475437,0.000000
7,Harrow,E09000015,5046.330,0.000,F,"POLYGON ((-0.40405 51.61318, -0.40318 51.61318...","1,050",235,466584.027,2780,24093.747,0.000000,0.000000,0.000000
8,Brent,E09000005,4323.270,0.000,F,"POLYGON ((-0.19654 51.52766, -0.19682 51.52774...",0,0,493454.700,2997,26531.182,35285.383766,27212.605551,8072.778215
9,Barnet,E09000003,8674.837,0.000,F,"POLYGON ((-0.19987 51.67017, -0.19968 51.66986...",809,80,811565.232,4614,41412.155,1407.774758,0.000000,1407.774758


In [52]:
# merge data sets
df = london_reprojected
df = pd.merge(df, road_stats_df, left_on="NAME", right_on="NAME", how="left")

In [ ]:
# calculate cycle routes ratio of total street network
df["TfL_cycle_ratio"] = df["TfL_cycle_routes_length_total"] / df["street_length_total"]